In [ ]:
import os
import pandas as pd
import yaml
import psycopg2
import logging
from fuzzywuzzy import process
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

# Configuração do logger
logging.basicConfig(level=logging.ERROR)

# Função para substituir valores nulos
def substitui_nulos(df):
    for coluna in df.columns:
        if df[coluna].dtype == 'object':  # Se a coluna for do tipo objeto (texto)
            moda = df[coluna].mode()[0]   # Calcula a moda
            df[coluna] = df[coluna].fillna(moda)  # Substitui nulos pela moda
        else:  # Se a coluna for numérica
            mediana = df[coluna].median()  # Calcula a mediana
            df[coluna] = df[coluna].fillna(mediana)  # Substitui nulos pela mediana
    return df

# Função para corrigir erros de digitação utilizando fuzzy matching
def corrigir_erros_digitacao(df, coluna, lista_valida):
    for i, valor in enumerate(df[coluna]):
        valor_str = str(valor) if pd.notnull(valor) else valor

        if valor_str not in lista_valida and pd.notnull(valor_str):
            correcao = process.extractOne(valor_str, lista_valida)[0]
            df.at[i, coluna] = correcao
    return df

# Função para tratar outliers em uma coluna
def tratar_outliers(df, coluna, minimo, maximo):
    mediana = df[(df[coluna] >= minimo) & (df[coluna] <= maximo)][coluna].median()
    df[coluna] = df[coluna].apply(lambda x: mediana if x < minimo or x > maximo else x)
    return df

# Função para salvar os escaladores
def save_scalers(df, nome_colunas):
    objects_dir = r"D:\Repositories\Data-Science\Credit_Score_Novadrive\Data_Analysis_Cleaning\objects"
    
    for nome_coluna in nome_colunas:
        scaler = StandardScaler()
        df[nome_coluna] = scaler.fit_transform(df[[nome_coluna]])
        
        # Ajustando o caminho para salvar no diretório correto
        scaler_path = os.path.join(objects_dir, f"scaler_{nome_coluna}.joblib")
        joblib.dump(scaler, scaler_path)

    return df

# Função para salvar os codificadores (encoders)
def save_encoders(df, nome_colunas):
    objects_dir = r"D:\Repositories\Data-Science\Credit_Score_Novadrive\Data_Analysis_Cleaning\objects"
    
    for nome_coluna in nome_colunas:
        label_encoder = LabelEncoder()
        df[nome_coluna] = label_encoder.fit_transform(df[nome_coluna])
        
        # Ajustando o caminho para salvar no diretório correto
        encoder_path = os.path.join(objects_dir, f"labelencoder_{nome_coluna}.joblib")
        joblib.dump(label_encoder, encoder_path)

    return df

# Função para buscar dados do banco de dados, ajustar tipos e salvar em CSV
def fetch_data_from_db(sql_query, config_path, output_csv_path=None):
    """
    Função para buscar dados do banco de dados, ajustar tipos e salvar em CSV.
    """
    try:
        if not os.path.exists(config_path):
            raise FileNotFoundError(f"Arquivo de configuração não encontrado: {config_path}")

        # Lê o arquivo de configuração
        with open(config_path, 'r') as file:
            config = yaml.safe_load(file)

        # Conecta ao banco de dados
        with psycopg2.connect(
            dbname=config['database_config']['dbname'], 
            user=config['database_config']['user'], 
            password=config['database_config']['password'], 
            host=config['database_config']['host']
        ) as con:
            with con.cursor() as cursor:
                # Executa a consulta SQL
                cursor.execute(sql_query)
                data = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]

                # Cria um DataFrame a partir dos dados retornados
                df = pd.DataFrame(data, columns=columns)

                # Verifica se o DataFrame não está vazio
                if df.empty:
                    logging.warning("A consulta retornou nenhum dado.")
                else:
                    # Retorna o DataFrame sem processamento
                    print(f"Quantidade de linhas e colunas: {df.shape}")

                    # Salva os dados em um arquivo CSV, se o caminho for fornecido
                    if output_csv_path:
                        os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
                        df.to_csv(output_csv_path, index=False, encoding='utf-8')
                        print(f"Dados salvos em: {output_csv_path}")
                return df

    except psycopg2.Error as e:
        logging.error(f"Erro ao conectar ou executar a consulta no banco de dados: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro
    except Exception as e:
        logging.error(f"Erro inesperado: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

# Lendo a consulta SQL do arquivo
sql_query = ""
with open(r"D:\Repositories\Data-Science\Credit_Score_Novadrive\queries\data_denormalization.sql", "r", encoding="utf-8") as file:
    sql_query = file.read()

# Caminho do arquivo de configuração
config_path = r'D:\Repositories\Data-Science\Credit_Score_Novadrive\Data_Analysis_Cleaning\config.yaml'

# Buscando os dados do banco de dados
df = fetch_data_from_db(sql_query, config_path)

# Exibindo os dados no Jupyter Notebook
if not df.empty:
    display(df)
else:
    print("Nenhum dado retornado.")


Quantidade de linhas e colunas: (150, 13)


,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660.0,Outros,Ens.Médio,MuitoBom,58.0,0,Solteiro,VoyageRoamer,84623.00,350000.00,bom
1,Empresário,21,46557.0,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.00,500000.00,bom
2,Dentista,13,43939.0,Própria,Ens.Médio,Bom,22.0,0,Casado,DoubleDuty,127151.00,320000.00,ruim
3,Engenheiro,10,37262.0,Própria,Superior,Baixo,35.0,0,Divorciado,AgileXplorer,28767.00,250000.00,bom
4,Contador,6,52606.0,Própria,PósouMais,Justo,26.0,0,Casado,TrailConqueror,199564.00,400000.00,ruim
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Médico,36,47480.0,Própria,Superior,Bom,64.0,0,Divorciado,SpeedFury,217011.00,800000.00,bom
146,Advogado,39,20860.0,Alugada,Ens.Fundamental,Bom,37.0,0,Víuvo,DoubleDuty,139244.00,320000.00,ruim
147,Arquiteto,26,31394.0,Própria,PósouMais,MuitoBom,53.0,1,Divorciado,ElegantCruise,107035.00,300000.00,ruim
148,Médico,19,39769.0,None,Ens.Médio,Baixo,63.0,0,Solteiro,VoyageRoamer,54520.00,350000.00,bom


In [42]:
# Antes de rodar a função
df_before = df.copy()

# Rodando a função
substitui_nulos(df)

# Comparando antes e depois
print(df_before.isnull().sum())  # Antes
print(df.isnull().sum())         # Depois


profissao          0
tempoprofissao     0
renda              0
tiporesidencia     0
escolaridade       0
score              0
idade              0
dependentes        0
estadocivil        0
produto            0
valorsolicitado    0
valortotalbem      0
classe             0
dtype: int64
profissao          0
tempoprofissao     0
renda              0
tiporesidencia     0
escolaridade       0
score              0
idade              0
dependentes        0
estadocivil        0
produto            0
valorsolicitado    0
valortotalbem      0
classe             0
dtype: int64


C:\Users\tmada\AppData\Local\Temp\ipykernel_14220\2470638287.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[coluna].fillna(moda, inplace=True)
C:\Users\tmada\AppData\Local\Temp\ipykernel_14220\2470638287.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe